In [1]:
import logging
from pathlib import Path
import requests
from bs4 import BeautifulSoup

from typing import List, Set, Tuple

from liblakehouse import LakeHouse

import asyncio
import aiohttp

In [2]:
minio_connection = ""

In [3]:
# Configuração básica de logging
logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [4]:
site_url = "https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/precos-revenda-e-de-distribuicao-combustiveis/serie-historica-do-levantamento-de-precos"

catalog = "landing" 
schema = "anp"
volume = "serie_levantamento_precos"

allowed_extensions = [".xlsx", ".xlsb"]

In [5]:
def get_file_urls(site_url: str, allowed_extensions: List[str]) -> Set[Tuple[str, str]]:

    response = requests.get(site_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    file_urls = set()

    for link in soup.find_all("a", href=True):

        file_url = str(link.get("href"))

        if "mensal" in file_url:
            for allowed_extension in allowed_extensions:
                if file_url.endswith(allowed_extension, -5):
                    file_name = file_url.split("/")[-1]
                    file_urls.add((file_url, file_name))

    return file_urls

In [6]:
file_urls = get_file_urls(site_url, allowed_extensions)

In [ ]:
async def download_files(
    file_urls: Set[Tuple[str, str]],
    retries: int = 3,
    tmp_dir: str = "/tmp",
    chunked_size: int = 1024
):

    async def fetch(
        session: aiohttp.ClientSession,
        file_url: str,
        file_name: str,
        retries: int = retries,
        tmp_dir: str = tmp_dir,
        chunked_size: int = chunked_size,
    ):
        for attempt in range(retries + 1):
            async with session.get(file_url) as response:
                status = response.status

                if status != 200:
                    logger.error(
                        f"""
                        Attempt: [{attempt}/{retries}], 
                        File url: {file_url}, 
                        Error: mismatch status, expected: 200, got: {status}
                        """
                    )

                    continue

                path = Path(f"{tmp_dir}")
                path.mkdir(parents=True, exist_ok=True)
                dest_file_name = f"{path}/{file_name}"

                with open(dest_file_name, "wb") as bf:
                    async for chunk in response.content.iter_chunked(chunked_size):
                        bf.write(chunk)

                headers = response.headers
                content_type = headers.get("Content-Type")
                content_length = headers.get("Content-Length")

                content_types = (
                    "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
                    "application/vnd.ms-excel.sheet.binary.macroEnabled.12",
                )

                

            logger.info(
                f"""
                Attempt [{attempt}/{retries}], 
                Status: {status}, 
                Type: {content_type}, 
                Length: {content_length} Bytes, 
                File dest: {dest_file_name}
                """
            )

            break

    async with aiohttp.ClientSession() as session:
        coro_or_futures = [
            fetch(session, file_url, file_name) for file_url, file_name in file_urls
        ]
        await asyncio.gather(*coro_or_futures)

In [8]:
await download_files(
    file_urls,
    tmp_dir="/tmp/anp/serie_historica_levantamento_precos"
)

2025-09-08 09:43:11,748 - __main__ - INFO - 
                Attempt [0/3], 
                Status: 200, 
                Type: application/vnd.openxmlformats-officedocument.spreadsheetml.sheet, 
                Length: 86611, 
                File dest: /tmp/anp/serie_historica_levantamento_precos
                
2025-09-08 09:43:11,781 - __main__ - INFO - 
                Attempt [0/3], 
                Status: 200, 
                Type: application/vnd.openxmlformats-officedocument.spreadsheetml.sheet, 
                Length: 105444, 
                File dest: /tmp/anp/serie_historica_levantamento_precos
                
2025-09-08 09:43:12,018 - __main__ - INFO - 
                Attempt [0/3], 
                Status: 200, 
                Type: application/vnd.openxmlformats-officedocument.spreadsheetml.sheet, 
                Length: 497378, 
                File dest: /tmp/anp/serie_historica_levantamento_precos
                
2025-09-08 09:43:12,043 - __main__ - INFO - 

In [9]:
# p = Path("/tmp/anp/serie_historica_levantamento_precos")
# p.mkdir(parents=True, exist_ok=True)

In [10]:
# f"{p}/acb.txt"

In [11]:
            # except Exception as e:
            #     logger.error(
            #         f"[{attempt}/{retries}] downloading: {url}, error: {e}"
            #     )

            #     continue

In [12]:
# def baixar_arquivos_com_metadados(links, pasta_destino="download"):
#     """
#     Baixa arquivos de uma lista de links e retorna metadados.
#     """
#     os.makedirs(pasta_destino, exist_ok=True)
#     registros = []
    
#     logging.info(f"Iniciando o download de {len(links)} arquivos para '{pasta_destino}'...")

#     for link in links:
#         nome_arquivo = link.split("/")[-1]
#         caminho_arquivo = os.path.join(pasta_destino, nome_arquivo)
#         data_download = datetime.now().isoformat()

#         try:
#             resposta = requests.get(link)
#             if resposta.status_code == 200:
#                 with open(caminho_arquivo, "wb") as f:
#                     f.write(resposta.content)
#                 logging.info(f"Baixado: {nome_arquivo}")

#                 registros.append({
#                     "nome_arquivo": nome_arquivo,
#                     "link_origem": link,
#                     "caminho_local": caminho_arquivo,
#                     "data_download": data_download
#                 })
#             else:
#                 logging.warning(f"Erro ao baixar: {link} (Status {resposta.status_code})")
#         except Exception as e:
#             logging.error(f"Erro ao baixar {link} -> {e}", exc_info=True)

#         time.sleep(0.5) # Pausa para não sobrecarregar o servidor
    
#     logging.info(f"Download finalizado. {len(registros)} arquivos baixados com sucesso.")
#     return registros

In [13]:
# def upload_files_to_minio(registros_baixados):
#     """
#     Faz o upload de uma lista de arquivos locais para o MinIO.
#     """
#     logging.info(f"\nIniciando o upload de {len(registros_baixados)} arquivos para o bucket '{bucket_name}'...")
    
#     for registro in registros_baixados:
#         caminho_local = registro["caminho_local"]
#         nome_arquivo = registro["nome_arquivo"]
#         s3_caminho_destino = s3_prefix + nome_arquivo

#         try:
#             s3_client.fput_object(bucket_name, s3_caminho_destino, caminho_local)
#             logging.info(f"Upload bem-sucedido: {caminho_local} -> s3a://{bucket_name}/{s3_caminho_destino}")
#         except Exception as e:
#             logging.error(f"Erro ao fazer o upload do arquivo {nome_arquivo}: {e}")

#     logging.info("Upload de arquivos finalizado.")

In [14]:
# # 1. Extrair URLs
# links_de_arquivos = extrair_urls_arquivos(url, extensoes_arquivos)

# # 2. Baixar arquivos
# if links_de_arquivos:
#     registros_baixados = baixar_arquivos_com_metadados(links_de_arquivos)

#     # 3. Fazer o upload e, em seguida, excluir
#     if registros_baixados:
#         try:
#             upload_files_to_minio(registros_baixados)
#             logging.info("Processo de extração, download e upload finalizado com sucesso.")
#         finally:
#             for registro in registros_baixados:
#                 os.remove(registro['caminho_local'])
#                 logging.info(f"Arquivo temporário excluído: {registro['caminho_local']}")
#     else:
#         logging.warning("Nenhum arquivo foi baixado. O processo de upload não foi iniciado.")
# else:
#     logging.warning("Nenhum link de arquivo encontrado. O processo foi encerrado.")